In [629]:
import pandas as pd
import numpy as np
import requests

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import plotly.graph_objects as go

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Lambda
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K

# Wczytanie danych z pliku CSV
file_path = '/root/Documents/dane_ok.csv'
df = pd.read_csv(file_path, delimiter='\t', encoding='ISO-8859-2')

movie_ids = df['Movie'].unique()
movie_id_mapping = {id: i for i, id in enumerate(movie_ids)}

# Pobranie ocen za pomocą ocen z OMDB_API

In [ ]:
API_KEY = "HEHE"

def get_movie_rating_from_api(movie_title):
    """
    Pobiera średnią ocenę filmu z OMDb API.
    """
    url = f"http://www.omdbapi.com/?t={movie_title}&apikey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'imdbRating' in data and data['imdbRating'] != "N/A":
            return float(data['imdbRating'])
    return None

movie_ratings = {}
for movie_id in movie_ids:
    movie_name = movie_id
    rating = get_movie_rating_from_api(movie_name)
    if rating is not None:
        movie_ratings[movie_id] = rating
        print(f"Średnia ocena z API dla '{movie_name}': {rating}")

ratings_df = pd.DataFrame(list(movie_ratings.items()), columns=['Movie', 'API_Rating'])
ratings_df = ratings_df.dropna(subset=['API_Rating'])
ratings_df.to_csv('/root/Documents/movie_ratings.csv', sep='\t', encoding='ISO-8859-2', index=False)

print("Średnie oceny zostały zapisane do pliku.")

# Uzupełnienie brakujących danych
Uzupełnienie brakujących ocen ocenami z API, lub średnią ocen użytkowników z CSV.

In [634]:
ratings_df = pd.read_csv('/root/Documents/movie_ratings.csv', delimiter='\t', encoding='ISO-8859-2')

def calculate_fallback_rating(movie_id, df):
    """
    Oblicza średnią ocenę filmu na podstawie danych użytkowników z CSV.
    Jeśli oceniał tylko jeden użytkownik, zwraca ocenę 5.
    """
    movie_ratings = df[df['Movie'] == movie_id]['Rating']
    if len(movie_ratings) == 1:
        return 5
    return movie_ratings.mean()

def fill_missing_ratings(df, ratings_df):
    """
    Uzupełnia brakujące oceny filmów dla każdego użytkownika.
    """
    ratings_map = dict(zip(ratings_df['Movie'], ratings_df['API_Rating']))

    # Tworzenie pełnej macierzy użytkownik-film
    full_data = []
    for user_id in df['User'].unique():
        for movie_id in df['Movie'].unique():
            existing_rating = df[(df['User'] == user_id) & (df['Movie'] == movie_id)]['Rating']
            if not existing_rating.empty:
                full_data.append({'User': user_id, 'Movie': movie_id, 'Rating': existing_rating.values[0]})
            else:
                # Jeśli brak oceny, używamy średniej z API lub CSV
                fallback_rating = ratings_map.get(movie_id) or calculate_fallback_rating(movie_id, df)
                full_data.append({'User': user_id, 'Movie': movie_id, 'Rating': fallback_rating})

    return pd.DataFrame(full_data)

# Uzupełnianie danych
df_filled = fill_missing_ratings(df, ratings_df)

# Skalowanie ocen do odległości w skali [0.0, 0.9]
df_filled['Distance'] = 1.0 - (df_filled['Rating'] / 10.0)

# Tworzenie unikalnego mapowania dla użytkowników (imiona -> liczby)
user_names = df_filled['User'].unique()
movie_names = df_filled['Movie'].unique()

user_id_mapping = {id: i for i, id in enumerate(user_names)}
movie_id_mapping = {id: i for i, id in enumerate(movie_names)}

user_name_to_id = {name: i for i, name in enumerate(user_names)}
movie_name_to_id = {name: i for i, name in enumerate(movie_names)}

# Zmiana kolumny 'User' na wartości liczbowe
df_filled['User'] = df_filled['User'].map(user_name_to_id)
df_filled['Movie'] = df_filled['Movie'].map(movie_name_to_id)

In [635]:
# Dane wejściowe
users = np.array(df_filled['User'].values, dtype=np.int32)
movies = np.array(df_filled['Movie'].values, dtype=np.int32)
distances = np.array(df_filled['Distance'].values, dtype=np.float32)

print("Dane zostały przygotowane.")

Dane zostały przygotowane.


In [636]:
print("Minimalna wartość users:", np.min(users))
print("Maksymalna wartość users:", np.max(users))

print("Minimalna wartość movies:", np.min(movies))
print("Maksymalna wartość movies:", np.max(movies))

print("Minimalna wartość distances:", np.min(distances))
print("Maksymalna wartość distances:", np.max(distances))

print("Ilość NaN:", df_filled.isna().sum())

Minimalna wartość users: 0
Maksymalna wartość users: 8
Minimalna wartość movies: 0
Maksymalna wartość movies: 250
Minimalna wartość distances: 0.0
Maksymalna wartość distances: 0.9
Ilość NaN: User        0
Movie       0
Rating      0
Distance    0
dtype: int64


# Podział zbioru na treningowy i testowy
W naszym przypadku zbiór testowy będzie też zbiorem walidacyjnym.

In [627]:
# Podział na zbiór treningowy i testowy
train_users, test_users, train_movies, test_movies, train_distances, test_distances = train_test_split(
    users, movies, distances, test_size=0.2, random_state=42
)

# Wyświetlenie rozmiarów danych
print(f"Liczba próbek w zbiorze treningowym: {len(train_users)}")
print(f"Liczba próbek w zbiorze testowym: {len(test_users)}")

Liczba próbek w zbiorze treningowym: 1807
Liczba próbek w zbiorze testowym: 452


# Defninicja modelu
Cel modelu to obliczanie odległości między użytkownikiem a filmem, odlęgłość reprezentuje ocenę. Im bliżej film jest użytkownika tym jest bardziej lubiany.

In [610]:

# Definicja modelu
embedding_size = 4

# Warstwy embedding
user_input = Input(shape=(1,), name="User_Input")
movie_input = Input(shape=(1,), name="Movie_Input")

user_embedding = Embedding(input_dim=num_users, output_dim=embedding_size, name="User_Embedding")(user_input)
movie_embedding = Embedding(input_dim=num_movies, output_dim=embedding_size, name="Movie_Embedding")(movie_input)

# Spłaszczenie embeddingów
user_vec = Flatten(name="Flatten_User")(user_embedding)
movie_vec = Flatten(name="Flatten_Movie")(movie_embedding)

# Obliczanie różnicy między embeddingami użytkownika i filmu
diff_vec = Subtract(name="Subtract")([user_vec, movie_vec])

# Podniesienie różnic do kwadratu (element-wise)
squared_diff = Multiply(name="Squared_Diff")([diff_vec, diff_vec])

# Suma wszystkich kwadratów
sum_squared_diff = Lambda(
    lambda x: K.sum(x, axis=1, keepdims=True), 
    output_shape=(1,),  # Kształt wyjścia po sumowaniu
    name="Sum_Squared_Diff"
)(squared_diff)


euclidean_distance = Lambda(
    lambda x: K.sqrt(x), 
    output_shape=(1,), 
    name="Euclidean_Distance"
)(sum_squared_diff)


# Model
model = Model(inputs=[user_input, movie_input], outputs=euclidean_distance)
model.compile(optimizer=Adam(learning_rate=0.0001), loss="mse")

# Uczenie modelu
Uczenie modelu z wczesnym przerwaniem gdy strata na zbiorze walidacyjnym będzie minimalna.

In [611]:

# Dodanie callbacka EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',       # Monitorowana metryka
    patience=32,               # Liczba epok bez poprawy przed zatrzymaniem
    restore_best_weights=True # Przywrócenie najlepszych wag modelu
)

# Trenowanie modelu z walidacją i EarlyStopping
history = model.fit(
    [train_users, train_movies],
    train_distances,
    batch_size=32,
    validation_data=([test_users, test_movies], test_distances),
    epochs=1000,
    verbose=1,
    callbacks=[early_stopping]
)

Epoch 1/1000
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1187 - val_loss: 0.1180
Epoch 2/1000
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1081 - val_loss: 0.1126
Epoch 3/1000
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1078 - val_loss: 0.1070
Epoch 4/1000
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0993 - val_loss: 0.1011
Epoch 5/1000
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0937 - val_loss: 0.0954
Epoch 6/1000
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0919 - val_loss: 0.0898
Epoch 7/1000
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0818 - val_loss: 0.0844
Epoch 8/1000
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0762 - val_loss: 0.0794
Epoch 9/1000
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0718 - val_loss: 0.0747
Epoch 10/1000
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0689 - val_loss: 0.0703
Epoch 11/1000
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0618 - val_loss: 0.0662
Epoch 12/1000
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

# Wizualizacja
Wizualizacja wyników w przestrzeni 3D, zmniejszenie wymiarowości osadzeń za pomocą PCA

In [637]:

# Pobranie embeddingów z wytrenowanego modelu
user_embedding_layer = model.get_layer("User_Embedding")
movie_embedding_layer = model.get_layer("Movie_Embedding")

# Pobranie wag embeddingów
user_embeddings = user_embedding_layer.get_weights()[0]  # Macierz użytkowników
movie_embeddings = movie_embedding_layer.get_weights()[0]  # Macierz filmów

# Redukcja wymiarów do 3D za pomocą PCA
pca = PCA(n_components=3)
user_embeddings_3d = pca.fit_transform(user_embeddings)
movie_embeddings_3d = pca.transform(movie_embeddings)

# Mapowanie nazw użytkowników i filmów
user_names = {v: k for k, v in user_id_mapping.items()}  # Odwrócenie mapowania dla użytkowników
movie_names = {v: k for k, v in movie_id_mapping.items()}  # Odwrócenie mapowania dla filmów

# Wizualizacja za pomocą Plotly
fig = go.Figure()

# Dodanie użytkowników do wykresu
fig.add_trace(go.Scatter3d(
    x=user_embeddings_3d[:, 0],
    y=user_embeddings_3d[:, 1],
    z=user_embeddings_3d[:, 2],
    mode='markers+text',
    marker=dict(size=8, color='red'),
    text=[user_names[i] for i in range(len(user_names))],  # Nazwy użytkowników
    name='Users'
))

# Dodanie filmów do wykresu
fig.add_trace(go.Scatter3d(
    x=movie_embeddings_3d[:, 0],
    y=movie_embeddings_3d[:, 1],
    z=movie_embeddings_3d[:, 2],
    mode='markers',
    marker=dict(size=6, color='blue', opacity=0.6),
    text=[movie_names[i] for i in range(len(movie_names))],  # Nazwy filmów
    name='Movies',
    hoverinfo='text'
))

# Ustawienia wykresu
fig.update_layout(
    title="Wizualizacja embeddingów użytkowników i filmów w 3D",
    scene=dict(
        xaxis_title="Wymiar 1",
        yaxis_title="Wymiar 2",
        zaxis_title="Wymiar 3"
    ),
    showlegend=True,
    height=800  # Wysokość okna wykresu
)

# Wyświetlenie wykresu
fig.show()


In [615]:
def recommend_movies_for_user(user_id, user_embeddings, movie_embeddings, movie_names, user_id_mapping, df, top_n=5, best=True):
    """
    Proponuje najlepsze filmy dla danego użytkownika na podstawie embeddingów.
    
    Parameters:
    - user_id: ID użytkownika (oryginalny ID, nie indeks).
    - user_embeddings: Macierz embeddingów użytkowników.
    - movie_embeddings: Macierz embeddingów filmów.
    - movie_names: Słownik mapujący indeksy na nazwy filmów.
    - user_id_mapping: Słownik mapujący ID użytkownika na indeks w embeddingach.
    - df: DataFrame z kolumnami ['User', 'Movie'], wskazujący, które filmy użytkownik ocenił.
    - top_n: Liczba filmów do zarekomendowania (domyślnie 5).
    
    Returns:
    - Lista (nazwa filmu, odległość) dla najlepszych filmów.
    """
    # Pobranie indeksu użytkownika w macierzy embeddingów
    user_idx = user_id_mapping[user_id]
    
    # Embedding użytkownika
    user_vector = user_embeddings[user_idx]
    
    # Lista filmów ocenionych przez użytkownika (konwersja nazw na indeksy)
    rated_movies = df[df['User'] == user_id]['Movie'].values
    rated_movies_indices = [movie_name_to_id[movie] for movie in rated_movies if movie in movie_name_to_id]

    # Obliczenie odległości euklidesowej między użytkownikiem a każdym filmem
    distances = np.linalg.norm(movie_embeddings - user_vector, axis=1)
    
    # Posortowanie filmów według odległości (najmniejsze odległości jako pierwsze)
    if best: 
        sorted_indices = np.argsort(distances)
    else:
        sorted_indices = np.argsort(-distances)
    
    # Filtrowanie filmów ocenionych przez użytkownika
    recommendations = [(movie_names[i], distances[i]) for i in sorted_indices if i not in rated_movies_indices]
    
    # Pobranie najlepszych filmów (top_n)
    return recommendations[:top_n]


In [642]:
# Przykład użycia:
user_id = 'Paweł Cz.'
top_n = 5    # Liczba rekomendacji

# DataFrame `df` zawiera oceny użytkowników
recommendations = recommend_movies_for_user(
    user_id=user_id,
    user_embeddings=user_embeddings,
    movie_embeddings=movie_embeddings,
    movie_names={v: k for k, v in movie_id_mapping.items()}, 
    user_id_mapping=user_id_mapping,
    df=df,
    top_n=top_n
)

recommendations2 = recommend_movies_for_user(
    user_id=user_id,
    user_embeddings=user_embeddings,
    movie_embeddings=movie_embeddings,
    movie_names={v: k for k, v in movie_id_mapping.items()},
    user_id_mapping=user_id_mapping,
    df=df,
    top_n=top_n,
    best=False
)


# Wyświetlenie rekomendacji
print(f"Top 5 filmów dla użytkownika {user_id}:")
for movie, distance in recommendations:
    print(f"ocena: {((1-distance)*10.0):.4f}\t[{movie}]")
    
print(f"\n")

print(f"Top 5 kinematograficznego ścieku dla użytkownika {user_id}:")
for movie, distance in recommendations2:
    print(f"ocena: {((1-distance)*10.0):.4f}\t[{movie}]")

Top 5 filmów dla użytkownika Paweł Cz.:
ocena: 8.9501	[Wilk z Wall Street]
ocena: 8.8133	[Arcane]
ocena: 8.7911	[Harry Potter]
ocena: 8.7668	[Breaking Bad]
ocena: 8.6750	[Hobbit]


Top 5 kinematograficznego ścieku dla użytkownika Paweł Cz.:
ocena: 1.3136	[Smoleńsk]
ocena: 2.4328	[50 twarzy greya]
ocena: 3.4871	[Kevin sam w domu]
ocena: 3.8398	[Botoks]
ocena: 4.6236	[Puchatek: Krew i miód]


In [643]:
# Przykład użycia:
user_id = 'Karol Sz.'
top_n = 5    # Liczba rekomendacji

# DataFrame `df` zawiera oceny użytkowników
recommendations = recommend_movies_for_user(
    user_id=user_id,
    user_embeddings=user_embeddings,
    movie_embeddings=movie_embeddings,
    movie_names={v: k for k, v in movie_id_mapping.items()}, 
    user_id_mapping=user_id_mapping,
    df=df,
    top_n=top_n
)

recommendations2 = recommend_movies_for_user(
    user_id=user_id,
    user_embeddings=user_embeddings,
    movie_embeddings=movie_embeddings,
    movie_names={v: k for k, v in movie_id_mapping.items()},
    user_id_mapping=user_id_mapping,
    df=df,
    top_n=top_n,
    best=False
)


# Wyświetlenie rekomendacji
print(f"Top 5 filmów dla użytkownika {user_id}:")
for movie, distance in recommendations:
    print(f"ocena: {((1-distance)*10.0):.4f}\t[{movie}]")
    
print(f"\n")

print(f"Top 5 kinematograficznego ścieku dla użytkownika {user_id}:")
for movie, distance in recommendations2:
    print(f"ocena: {((1-distance)*10.0):.4f}\t[{movie}]")

Top 5 filmów dla użytkownika Karol Sz.:
ocena: 8.8007	[The Boys]
ocena: 8.6493	[Zielona Mila]
ocena: 8.5955	[Skazani na Shawshank]
ocena: 8.5855	[Grand Budapest Hotel]
ocena: 8.5529	[Gladiator]


Top 5 kinematograficznego ścieku dla użytkownika Karol Sz.:
ocena: 2.2777	[Smoleńsk]
ocena: 2.6979	[Wyjazd integracyjny]
ocena: 4.4971	[Iluzja]
ocena: 4.5422	[Substancja]
ocena: 4.6692	[Kapitan Ameryka: Wojna bohaterów]


In [644]:
# Przykład użycia:
user_id = 'Michał J.'
top_n = 5    # Liczba rekomendacji

# DataFrame `df` zawiera oceny użytkowników
recommendations = recommend_movies_for_user(
    user_id=user_id,
    user_embeddings=user_embeddings,
    movie_embeddings=movie_embeddings,
    movie_names={v: k for k, v in movie_id_mapping.items()}, 
    user_id_mapping=user_id_mapping,
    df=df,
    top_n=top_n
)

recommendations2 = recommend_movies_for_user(
    user_id=user_id,
    user_embeddings=user_embeddings,
    movie_embeddings=movie_embeddings,
    movie_names={v: k for k, v in movie_id_mapping.items()},
    user_id_mapping=user_id_mapping,
    df=df,
    top_n=top_n,
    best=False
)


# Wyświetlenie rekomendacji
print(f"Top 5 filmów dla użytkownika {user_id}:")
for movie, distance in recommendations:
    print(f"ocena: {((1-distance)*10.0):.4f}\t[{movie}]")
    
print(f"\n")

print(f"Top 5 kinematograficznego ścieku dla użytkownika {user_id}:")
for movie, distance in recommendations2:
    print(f"ocena: {((1-distance)*10.0):.4f}\t[{movie}]")

Top 5 filmów dla użytkownika Michał J.:
ocena: 8.8841	[Sherlock]
ocena: 8.7362	[Yellowstone]
ocena: 8.7235	[The Last of Us]
ocena: 8.7092	[Wyspa tajemnic]
ocena: 8.6572	[Kapitan Bomba]


Top 5 kinematograficznego ścieku dla użytkownika Michał J.:
ocena: 2.3336	[50 twarzy greya]
ocena: 2.5706	[Lucyfer]
ocena: 2.7838	[Wyjazd integracyjny]
ocena: 3.5499	[Kevin sam w domu]
ocena: 3.9740	[Botoks]
